In [ ]:
!pip install -q transformers sentence-transformers faiss-cpu pandas numpy tqdm requests

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

os.makedirs('/root/.kaggle', exist_ok=True)
os.rename('kaggle.json', '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 600)

In [ ]:
download_path = "/content/drive/My Drive/lyrics_project"
os.makedirs(download_path, exist_ok=True)

!kaggle datasets download -d nikhilnayak123/5-million-song-lyrics-dataset -p "$download_path"

Dataset URL: https://www.kaggle.com/datasets/nikhilnayak123/5-million-song-lyrics-dataset
License(s): unknown
5-million-song-lyrics-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile

zip_file_path = f"{download_path}/5-million-song-lyrics-dataset.zip"
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(download_path)

In [ ]:
# import pandas as pd

# csv_path = f"{download_path}/ds2.csv"  # Adjust if needed
# df = pd.read_csv(csv_path)
# df = df[['song', 'artist', 'lyrics']].dropna()
# df = df[df['lyrics'].str.len() > 100].head(10000)  # Load top 10k
# df.head()


import pandas as pd

csv_path = f"{download_path}/ds2.csv"
chunksize = 10000  # Load 10k rows at a time
target_rows = []

for chunk in pd.read_csv(csv_path, usecols=['title', 'artist', 'lyrics'], chunksize=chunksize):
    chunk = chunk.dropna()
    chunk = chunk[chunk['lyrics'].str.len() > 100]
    target_rows.append(chunk)
    if sum(len(x) for x in target_rows) >= 10000:
        break

df = pd.concat(target_rows).head(10000)
df.reset_index(drop=True, inplace=True)
df.head()


,title,artist,lyrics
0,Killa Cam,Cam'ron,"[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki..."
1,Can I Live,JAY-Z,"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,..."
2,Forgive Me Father,Fabolous,Maybe cause I'm eatin\nAnd these bastards fien...
3,Down and Out,Cam'ron,[Produced by Kanye West and Brian Miller]\n\n[...
4,Fly In,Lil Wayne,"[Intro]\nSo they ask me\n""Young boy\nWhat you ..."


In [ ]:
!pip install -q sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

model = SentenceTransformer('all-MiniLM-L6-v2')  # Light & effective
tqdm.pandas()

# Add embeddings column (this might take a few minutes)
df['embedding'] = df['lyrics'].progress_apply(lambda x: model.encode(x, show_progress_bar=False))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

100%|██████████| 10000/10000 [31:12<00:00,  5.34it/s]


In [ ]:
!pip install -q faiss-cpu

In [ ]:
import numpy as np

# Convert embeddings to numpy matrix
embedding_matrix = np.vstack(df['embedding'].values).astype('float32')

In [ ]:
import faiss

index = faiss.IndexFlatL2(embedding_matrix.shape[1])  # L2 distance (cosine is optional alternative)
index.add(embedding_matrix)


In [ ]:
def generate_playlist(prompt, k=5):
    query_vec = model.encode(prompt).astype('float32')
    D, I = index.search(np.array([query_vec]), k)
    results = df.iloc[I[0]][['title', 'artist', 'lyrics']]
    return results


In [ ]:
playlist = generate_playlist("doing a workout at the gym", k=10)
playlist

,title,artist,lyrics
7240,Exercise,Akinyele,[heavy breathing ]\nCheck this out man\nAll th...
1035,The New Workout Plan,Kanye West,[Intro]\nYou just popped in the Kanye West\nGe...
673,Dont Sweat the Technique,Eric B. & Rakim,[Hook]\nDon't sweat the technique\nDon't sweat...
2746,6 Foot 7 Foot Freestyle,King Los,"Okay my work out devastating, it petrify hater..."
5350,Lean Wit It Rock Wit It,Dem Franchize Boyz,"[Repeat 8X]\nLean wit it, rock wit it!\n\n[Ver..."
7421,U Cant Fuck Wit It,Akrobatik,"Challenge my verbal gymnastics\n\nYo my style,..."
4032,Move That Body,Nelly,"[Hook: T-Pain]\nMove, that body\nWork, that bo..."
4862,Rock That Body,Black Eyed Peas,[Intro: Rob Base]\nI wanna rock right now\nI w...
8017,Perfectionist,Asher Roth,"[Intro]\nWorldwide\nAsher Roth, Perfection, Ro..."
3459,Double Up,R. Kelly,"[Intro: R. Kelly]\nAight, now we got that out ..."


In [ ]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
import gradio as gr

# def generate_playlist_gradio(prompt):
#     query_vec = model.encode(prompt).astype('float32')
#     D, I = index.search(np.array([query_vec]), 5)
#     results = df.iloc[I[0]]

#     playlist_output = ""
#     for i, row in results.iterrows():
#         playlist_output += f"🎵 **{row['title']}** by *{row['artist']}*\n\n"
#         playlist_output += f"📝 *Lyrics preview*: `{row['lyrics'][:150]}...`\n\n"
#         playlist_output += "---\n"

#     return playlist_output
moods = [
    "breakup",
    "late night driving",
    "gym motivation",
    "deep focus",
    "studying on a rainy day",
    "pre-game hype",
    "healing from heartbreak",
    "sunset beach walk"
]

def generate_playlist_gradio(mood_prompt):
    query_vec = model.encode(mood_prompt).astype('float32')
    D, I = index.search(np.array([query_vec]), 5)
    results = df.iloc[I[0]]

    output = ""
    for _, row in results.iterrows():
        title = row['title']
        artist = row['artist']
        lyrics_preview = row['lyrics'][:150].replace("\n", " ").strip()

        # Generate YouTube and Spotify search links
        yt_link = f"https://www.youtube.com/results?search_query={title}+{artist}".replace(" ", "+")
        sp_link = f"https://open.spotify.com/search/{title} {artist}".replace(" ", "%20")

        output += f"🎵 **{title}** by *{artist}*\n"
        output += f"📝 *Lyrics preview:* `{lyrics_preview}...`\n"
        output += f"🔗 [YouTube Search]({yt_link}) | [Spotify Search]({sp_link})\n\n"
        output += "---\n"

    return output


In [ ]:
import gradio as gr

gr.Interface(
    fn=generate_playlist_gradio,
    inputs=gr.Dropdown(choices=moods, label="Choose a Mood or Situation"),
    outputs="markdown",
    title="🎶 AI Playlist Generator",
    description="Select a mood and get a 5-song playlist with lyrics previews and search links!"
).launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://438ece336d5d5983ef.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
